## User Satsfaction Score Prediction

In [1]:
import pandas as pd
import os
import sys
# Add the 'scripts' directory to the Python path for module imports
sys.path.append(os.path.abspath(os.path.join('..', 'scripts')))

In [2]:
# Load the xdr_cleaned dataset
data = pd.read_csv('../Data/data/xdr_cleaned.csv')

In [3]:
# Import modules
from user_engagement_analysis import UserEngagementAnalysis
from experience_analytics import ExperienceAnalytics
from satisfaction_analytics import UserSatisfactionAnalytics

In [4]:
# Create instances of engagement and user experience classes
engagement_analysis = UserEngagementAnalysis(data)
experience_analysis = ExperienceAnalytics(data)
satisfaction_analysis = UserSatisfactionAnalytics()


In [5]:
# Normalize and cluster
# Aggregate engagement metrics
engagement_analysis.aggregate_metrics()
engagement_analysis.metrics.head()
engagement_analysis.normalize_and_cluster(n_clusters=3)


In [10]:
# Computer Engagement Score

engagement_features = [
    'total_session_duration',
    'total_download_traffic',
    'total_upload_traffic', 
    'sessions_frequency']

engagement_scores = satisfaction_analysis.compute_score(
    engagement_analysis.metrics, 
    engagement_analysis.cluster_centers, 
    engagement_features, 'Engagement_Score')


In [13]:
engagement_scores

,MSISDN/Number,Engagement_Score
0,3.361370e+10,3.706190e+09
1,3.361370e+10,2.442693e+08
2,3.361370e+10,4.146567e+08
3,3.361370e+10,1.199990e+07
4,3.361370e+10,1.925831e+08
...,...,...
89326,3.376818e+10,3.330732e+08
89327,3.376818e+10,1.553796e+07
89328,3.376819e+10,3.989745e+08
89329,3.376819e+10,6.568295e+07


In [7]:
# User experience clusters
# Define features
experience_features = ['TCP Retransmission', 'RTT', 'Throughput']
experience_df, cluster_centers = experience_analysis.k_means_clustering(experience_features)

In [9]:
# Computer User experience scores

experience_scores = satisfaction_analysis.compute_score(
    experience_df, 
    cluster_centers, 
    experience_features, 'Experience_Score')

In [11]:
experience_scores

,MSISDN/Number,Experience_Score
0,3.361370e+10,9.145345e+05
1,3.361370e+10,3.180830e+06
2,3.361370e+10,1.122624e+06
3,3.361370e+10,1.122624e+06
4,3.361370e+10,5.277909e+06
...,...,...
89326,3.376818e+10,1.122624e+06
89327,3.376818e+10,1.122624e+06
89328,3.376819e+10,1.122624e+06
89329,3.376819e+10,5.150907e+06
